<a href="https://colab.research.google.com/github/hasiburrahman1/NLP/blob/master/complex_type_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np 
import pandas as pd 
import os
import re
import matplotlib.pyplot as plt
import xml.etree.ElementTree as et 
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, SimpleRNN, GRU, LSTM, Bidirectional, Dropout, Input, Conv2D, MaxPool2D
from keras.layers.embeddings import Embedding
from keras.initializers import Constant
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.optimizers import Adam
from keras.models import Model
from keras.layers.convolutional import Conv1D
from keras.layers import Conv1D, Dense, MaxPool1D, Flatten, Input, GlobalMaxPooling1D

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split
import tensorflow as tf
import collections
import unicodedata
import re
import numpy as np
import os
import io
import time
import xlrd
import csv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
'''
cols_to_use = ['Question', 'Answer'] # or [0,1,2,3]
df = pd.read_csv('data.csv', usecols= cols_to_use)
'''

"\ncols_to_use = ['Question', 'Answer'] # or [0,1,2,3]\ndf = pd.read_csv('data.csv', usecols= cols_to_use)\n"

In [ ]:
'''
mport pandas
df = pandas.read_excel(r"/test.xlsx")
df = df.drop('label', 1)
#print the column names
print(df.columns)

#get the values for a given column
#values = df['collumn_name'].values

#get a data frame with selected columns
FORMAT = ['Question', 'Answer']
df_selected = df[FORMAT]


df.to_csv('dataset_csv.csv')
!cp dataset_csv.csv "drive/My Drive/"

df.describe()
'''

'\nmport pandas\ndf = pandas.read_excel(r"/test.xlsx")\ndf = df.drop(\'label\', 1)\n#print the column names\nprint(df.columns)\n\n#get the values for a given column\n#values = df[\'collumn_name\'].values\n\n#get a data frame with selected columns\nFORMAT = [\'Question\', \'Answer\']\ndf_selected = df[FORMAT]\n\n\ndf.to_csv(\'dataset_csv.csv\')\n!cp dataset_csv.csv "drive/My Drive/"\n\ndf.describe()\n'

---------------------------**Start**-------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# dataset path
#path_to_file = "/datasetCleaned.txt"
path_to_file = "/content/drive/My Drive/Colab Notebooks/dataset/test_Text_Document.txt"

In [ ]:
def preprocess_sentence(w):
#     remove punctuations from lines
    w = re.sub(r"([?.!,¿।])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)
    
    w = w.rstrip().strip()
    #print(w)
    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '<start> ' + w + ' <end>'
    #print(w)
    return w

In [ ]:
def vocabularyCounter(dataQuestion, dataAnswer):
    # setup Vocabulary 
    question_words = collections.Counter([word for sentence in dataQuestion for word in sentence.split()])
    answer_words = collections.Counter([word for sentence in dataAnswer for word in sentence.split()])
    
    print('Total English words: {}'.format(len([word for sentence in dataQuestion for word in sentence.split()])))
    print('Unique English words: {}'.format(len(question_words)))
    print('Most Common Words: "' + '" "'.join(list(zip(*question_words.most_common(10)))[0]) + '"\n')

    print('Total Bangla words: {}'.format(len([word for sentence in dataAnswer for word in sentence.split()])))
    print('Unique Bangla words: {}'.format(len(answer_words)))
    print('Most Common Words: "' + '" "'.join(list(zip(*answer_words.most_common(10)))[0]) + '"')

In [ ]:

# 1. Remove the accents
# 2. Clean the sentences
# 3. Return word pairs in the format: [ENGLISH, BANGLA]
def create_dataset(path, num_examples):
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')
    
    """
    test test test
    """
    #print(lines)
    #print("\n")
    #print(num_examples)
    """
    test test test
    """

    question = []
    answer = []
    for l in lines[:num_examples]:
        #print(l)
        w = l.split('\t')
        """
        test test test
        """
        #print(w)
        #print("\n")
        """
        test test test
        """
        question.append(w[0])
        answer.append(w[1])
    vocabularyCounter(question, answer) 
    word_pairs = [[preprocess_sentence(w) for w in l.split('\t')]  for l in lines[:num_examples]]
    #print(word_pairs)
#     rearrange dataset src and target
    for indd in range(len(word_pairs)):
        temp = word_pairs[indd][0]
        word_pairs[indd][0] = word_pairs[indd][1]
        word_pairs[indd][1] = temp
    print(word_pairs)
    return zip(*word_pairs)


In [ ]:
# preprocessing function declear and vocabulary checking
bn, en = create_dataset(path_to_file, None)

print("\n\n", en[4])
print(bn)

Total English words: 40
Unique English words: 9
Most Common Words: "Do" "lincRNAs" "play" "a" "role" "human" "cancer?" "1in" "in"

Total Bangla words: 66
Unique Bangla words: 50
Most Common Words: "lincRNAs" "of" "and" "to" "polyadenylation" "in" "cancer" "we" "the" "Genome-wide"
[['<start> Genome-wide identification and predictive modeling of lincRNAs polyadenylation in cancer genome . <end>', '<start> Do lincRNAs play a role in human cancer ? <end>'], ['<start> As expected we found lincRNAs to be present in CD14 monocytes of RA patients . <end>', '<start> Do lincRNAs play a role 1in human cancer ? <end>'], ['<start> Functional relevance of lincRNAs was further examined by biochemical assays . <end>', '<start> Do lincRNAs play a role 1in human cancer ? <end>'], ['<start> To date the mechanism of lincRNAs polyadenylation related to cancer is rarely fully_understood yet . <end>', '<start> Do lincRNAs play a role 1in human cancer ? <end>'], ['<start> Finally we propose a genetic mediator